# Composite Pattern

Code repris et commenté depuis https://refactoring.guru/design-patterns/composite/python/example

## Structure du pattern

![Diagramme de fonctionnement du pattern Composite](Images/CompositePattern.PNG)

## Diagramme de classe de l'exemple 

![Diagramme de classe de l'exemple](Images/CompositePatternClass.PNG)

Le but est de pouvoir interagir avec un leaf ou un composite (plusieurs leaf) de la même façon.  
Pour cela, __Leaf__ et __Composite__ implémentent la même interface __Component__.  

Leaf et composite disposent donc d'une même méthode "operation", qui sera implémentée différemment pour Leaf et pour Composite (appel des méthodes "operation" pour chacune des leaf qui le composent).

In [2]:
from __future__ import annotations
from abc import ABC, abstractmethod
from typing import List

In [11]:
class Component(ABC):

    # Getter et setter pour définir un parent (de type Component)
    @property
    def parent(self) -> Component:
        return self._parent

    @parent.setter
    def parent(self, parent: Component):
        self._parent = parent

    # Dans certains cas peut être utile de définir ici une façon de gérer les enfants
    # Serait par contre inutile pour tous les composant Leaf
    # Voir notes en anglais dans le code original
    
    def add(self, component: Component) -> None:
        pass

    def remove(self, component: Component) -> None:
        pass

    def is_composite(self) -> bool:
        # Pour permettre au code client de savoir si Component peut recevoir des enfants.
        return False

    @abstractmethod
    def operation(self) -> str:
        # Peut implémenter un comportement par défaut ou être abstraite
        pass


In [12]:
class Leaf(Component):
    # N'a pas d'enfants, est en général celui qui fait le boulot
    # (Composite délègue généralement à ses sous-composants)

    def operation(self) -> str:
        return "Leaf"

In [15]:
class Composite(Component):
    # Composant complexe pouvant avoir plusieurs enfants (leaf ou composite)
    # Délègue le travail aux enfants, compile et retourne le résultat.

    def __init__(self) -> None:
        self._children: List[Component] = []

    """
    A composite object can add or remove other components (both simple or
    complex) to or from its child list.
    """
    # Peut s'ajouter ou supprimer des components 
    def add(self, component: Component) -> None:
        self._children.append(component)
        component.parent = self

    def remove(self, component: Component) -> None:
        self._children.remove(component)
        component.parent = None

    # Indique qu'il est composite
    def is_composite(self) -> bool:
        return True

    def operation(self) -> str:
        # Appel récursif parmi ses enfants (Composites ou Leaf)
        # Collecte, somme et renvoie les résultats

        results = []
        for child in self._children:
            results.append(child.operation())
        return f"Branch({'+'.join(results)})"

On définit deux méthodes clientes (pas des classes!) prenant en paramètre un ou plusieurs composants et qui vont appeler "operation" sur un composant.

In [17]:
def client_code(component: Component) -> None:
    # Fonctionne avec l'interface de Component
    # Appelle "Operation" sur le Component passé en paramètres
    print(f"RESULT: {component.operation()}", end="")

In [18]:
def client_code2(component1: Component, component2: Component) -> None:
    # On fait le child manafement dans la classe Component (méthodes Add, Remove)
    # On ne dépend donc pas des classes concrètes pour construire l'arbre

    if component1.is_composite():
        component1.add(component2)

    print(f"RESULT: {component1.operation()}", end="")

## Code test

In [20]:
 # This way the client code can support the simple leaf components...
simple = Leaf()
print("Client: I've got a simple component:")
client_code(simple)
print("\n")

# ...as well as the complex composites.
tree = Composite()

branch1 = Composite()
branch1.add(Leaf())
branch1.add(Leaf())

branch2 = Composite()
branch2.add(Leaf())

tree.add(branch1)
tree.add(branch2)

print("Client: Now I've got a composite tree:")
client_code(tree)
print("\n")

print("Client: I don't need to check the components classes even when managing the tree:")
client_code2(tree, simple)

Client: I've got a simple component:
RESULT: Leaf

Client: Now I've got a composite tree:
RESULT: Branch(Branch(Leaf+Leaf)+Branch(Leaf))

Client: I don't need to check the components classes even when managing the tree:
RESULT: Branch(Branch(Leaf+Leaf)+Branch(Leaf)+Leaf)

## Code complet

```Python
from __future__ import annotations
from abc import ABC, abstractmethod
from typing import List


class Component(ABC):
    """
    The base Component class declares common operations for both simple and
    complex objects of a composition.
    """

    @property
    def parent(self) -> Component:
        return self._parent

    @parent.setter
    def parent(self, parent: Component):
        """
        Optionally, the base Component can declare an interface for setting and
        accessing a parent of the component in a tree structure. It can also
        provide some default implementation for these methods.
        """

        self._parent = parent

    """
    In some cases, it would be beneficial to define the child-management
    operations right in the base Component class. This way, you won't need to
    expose any concrete component classes to the client code, even during the
    object tree assembly. The downside is that these methods will be empty for
    the leaf-level components.
    """

    def add(self, component: Component) -> None:
        pass

    def remove(self, component: Component) -> None:
        pass

    def is_composite(self) -> bool:
        """
        You can provide a method that lets the client code figure out whether a
        component can bear children.
        """

        return False

    @abstractmethod
    def operation(self) -> str:
        """
        The base Component may implement some default behavior or leave it to
        concrete classes (by declaring the method containing the behavior as
        "abstract").
        """

        pass


class Leaf(Component):
    """
    The Leaf class represents the end objects of a composition. A leaf can't
    have any children.

    Usually, it's the Leaf objects that do the actual work, whereas Composite
    objects only delegate to their sub-components.
    """

    def operation(self) -> str:
        return "Leaf"


class Composite(Component):
    """
    The Composite class represents the complex components that may have
    children. Usually, the Composite objects delegate the actual work to their
    children and then "sum-up" the result.
    """

    def __init__(self) -> None:
        self._children: List[Component] = []

    """
    A composite object can add or remove other components (both simple or
    complex) to or from its child list.
    """

    def add(self, component: Component) -> None:
        self._children.append(component)
        component.parent = self

    def remove(self, component: Component) -> None:
        self._children.remove(component)
        component.parent = None

    def is_composite(self) -> bool:
        return True

    def operation(self) -> str:
        """
        The Composite executes its primary logic in a particular way. It
        traverses recursively through all its children, collecting and summing
        their results. Since the composite's children pass these calls to their
        children and so forth, the whole object tree is traversed as a result.
        """

        results = []
        for child in self._children:
            results.append(child.operation())
        return f"Branch({'+'.join(results)})"


def client_code(component: Component) -> None:
    """
    The client code works with all of the components via the base interface.
    """

    print(f"RESULT: {component.operation()}", end="")


def client_code2(component1: Component, component2: Component) -> None:
    """
    Thanks to the fact that the child-management operations are declared in the
    base Component class, the client code can work with any component, simple or
    complex, without depending on their concrete classes.
    """

    if component1.is_composite():
        component1.add(component2)

    print(f"RESULT: {component1.operation()}", end="")


if __name__ == "__main__":
    # This way the client code can support the simple leaf components...
    simple = Leaf()
    print("Client: I've got a simple component:")
    client_code(simple)
    print("\n")

    # ...as well as the complex composites.
    tree = Composite()

    branch1 = Composite()
    branch1.add(Leaf())
    branch1.add(Leaf())

    branch2 = Composite()
    branch2.add(Leaf())

    tree.add(branch1)
    tree.add(branch2)

    print("Client: Now I've got a composite tree:")
    client_code(tree)
    print("\n")

    print("Client: I don't need to check the components classes even when managing the tree:")
    client_code2(tree, simple)
```